# Titanic Data Cleaning Project
### Author: Tommaso Ocari
### Tools: Python, Pandas, Google Colab

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load the dataset

In [22]:
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Basic informations

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [24]:
# count the null per column
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [25]:
# number of duplicates
print(df.duplicated().sum())

0


## Remove duplicates

In [26]:
# remove duplicates
df = df.drop_duplicates()

## Clean and standardize text columns

In [27]:
# extract title and surname from the full name

df['Title'] = df['Name'].str.extract(r',\s*([^.]*)\.')
df['Surname'] = df['Name'].str.extract(r'([^.]*)\,')
df['Title'] = df['Title'].str.strip().str.lower()
df['Surname'] = df['Surname'].str.strip().str.lower()

In [28]:
df['Sex'] = df['Sex'].str.strip().str.lower()

In [29]:
df['Embarked'] = df['Embarked'].fillna('Unknown')
df['Embarked'] = df['Embarked'].str.upper()

In [30]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Surname
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,mr,braund
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,mrs,cumings
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,miss,heikkinen
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,mrs,futrelle
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,mr,allen


## Handle with missing values

In [31]:
df['Age'] = df.groupby('Pclass')['Age'].transform(
    lambda x: x.fillna(x.median())
)

In [32]:
df['Fare'] = df.groupby('Pclass')['Fare'].transform(
    lambda x: x.fillna(x.median())
)

In [33]:
df['Cabin'] = df['Cabin'].fillna("Unknown")

## Convert data types

In [34]:
df['Pclass'] = df['Pclass'].astype('category')
df['Survived'] = df['Survived'].astype('category')
df['Sex'] = df['Sex'].astype('category')
df['Embarked'] = df['Embarked'].astype('category')
df['Title'] = df['Title'].astype('category')

## Outlier detection using IQR

In [35]:
Q1 = df['Fare'].quantile(0.25)
Q3 = df['Fare'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR

size_before = len(df)

df = df[(df['Fare'] >= lower) & (df['Fare'] <= upper)]

print('%d rows removed as outlier'%(size_before - len(df)))

116 rows removed as outlier


## Final clean dataset export

In [36]:
df.to_csv("titanic_cleaned.csv", index=False)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Surname
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S,mr,braund
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,miss,heikkinen
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,mrs,futrelle
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S,mr,allen
5,6,0,3,"Moran, Mr. James",male,24.0,0,0,330877,8.4583,Unknown,Q,mr,moran


### Data Cleaning Summary — Titanic Dataset

- Removed duplicate rows  
- Extracted title from passenger names ("mr", "mrs", etc.)  
- Standardized text columns (sex, embarked, titles)  
- Converted multiple columns to categorical types  
- Imputed missing values using:
  - Age → median by Pclass
  - Fare → median by Pclass
  - Cabin → "Unknown"
  - Embarked → "Unknown"
- Detected & removed outliers in Fare using IQR  
- Exported final cleaned dataset as `titanic_cleaned.csv`

Dataset is now analysis-ready for EDA, predictive modeling, BI dashboards, or reporting.